In [ ]:
!pip install pandas requests tqdm openpyxl


In [ ]:
!pip install fuzzywuzzy

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import process

def get_pubchem_info(name):
    """Get PubChem info: CID, Title, IUPACName, CanonicalSMILES, InChIKey, Synonyms."""
    print(f"Get PubChem info for {name}: CID, Title, IUPACName, CanonicalSMILES, InChIKey, Synonyms.")
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/"
    try:
        # Get properties
        props = "CID,Title,IUPACName,CanonicalSMILES,InChIKey"
        prop_url = f"{base_url}{name}/property/{props}/JSON"
        prop_res = requests.get(prop_url, timeout=10)
        
        print("\n")
        print(prop_res)
        
        prop_res.raise_for_status()
        prop_data = prop_res.json()['PropertyTable']['Properties'][0]
        
        # Get synonyms
        syn_url = f"{base_url}{name}/synonyms/JSON"
        syn_res = requests.get(syn_url, timeout=10)
        syn_res.raise_for_status()
        syn_data = syn_res.json()
        synonyms = syn_data['InformationList']['Information'][0]['Synonym']
        
        print("\n")
        print( prop_data)
        return {
            "CID": prop_data.get("CID"),
            "CanonicalName": prop_data.get("Title"),
            "IUPACName": prop_data.get("IUPACName"),
            "CanonicalSMILES": prop_data.get("CanonicalSMILES"),
            "InChIKey": prop_data.get("InChIKey"),
            "Synonyms": synonyms
        }
    except:
        return None

def build_synonym_mapping(names):
    """Build mapping from synonyms to full PubChem info."""
    print("Build mapping from synonyms to full PubChem info.")
    mapping = {}
    for name in tqdm(names, desc="Querying PubChem"):
        info = get_pubchem_info(name)
        if info:
            for syn in info['Synonyms']:
                mapping[syn.lower()] = info
    return mapping

def fuzzy_lookup(name, mapping, threshold=85):
    """Find closest synonym match and return PubChem info."""
    print("Find closest synonym match and return PubChem info.")
    if not mapping:
        return None
    match, score = process.extractOne(name.lower(), mapping.keys())
    if score >= threshold:
        return mapping[match]
    return None

def normalize_with_details(df, column_name, mapping):
    """Replace excipient names with PubChem info."""
    print("Replace excipient names with PubChem info.")
    results = []
    for val in df[column_name]:
        val_str = str(val)
        info = mapping.get(val_str.lower())
        if not info:
            info = fuzzy_lookup(val_str, mapping)
        if info:
            results.append({
                column_name: val_str,
                "CID": info["CID"],
                "CanonicalName": info["Canonical_Name"],
                "IUPACName": info["IUPACName"],
                "CanonicalSMILES": info["Canonical_SMILES"],
                "InChIKey": info["InChIKey"]
            })
        else:
            results.append({
                column_name: val_str,
                "CID": None,
                "CanonicalName": None,
                "IUPACName": None,
                "CanonicalSMILES": None,
                "InChIKey": None
            })
    return pd.DataFrame(results)

# === MAIN SCRIPT ===
input_file = "excipients.xlsx"  # Your uploaded file
df = pd.read_excel(input_file)

column_name = df.columns[0]  # Assume first column is excipient names
unique_names = df[column_name].dropna().unique()

# Build mapping
mapping = build_synonym_mapping(unique_names)

# Normalize and get details
df_cleaned = normalize_with_details(df, column_name, mapping)

# Save
df_cleaned.to_excel("excipients_pubchem_info.xlsx", index=False)

print("✅ Done! Saved as 'excipients_pubchem_info.xlsx'")


In [2]:
import requests
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import process

def get_pubchem_data(name):
    """Get PubChem canonical name and synonyms for a given excipient name."""
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/"
    
    try:
        # Get canonical name (Title)
        prop_url = f"{base_url}{name}/property/Title/JSON"
        prop_res = requests.get(prop_url, timeout=10)
        prop_res.raise_for_status()
        prop_data = prop_res.json()
        canonical_name = prop_data['PropertyTable']['Properties'][0]['Title']
        
        # Get synonyms
        syn_url = f"{base_url}{name}/synonyms/JSON"
        syn_res = requests.get(syn_url, timeout=10)
        syn_res.raise_for_status()
        syn_data = syn_res.json()
        synonyms = syn_data['InformationList']['Information'][0]['Synonym']
        
        return canonical_name, synonyms
    except:
        return None, []

def build_synonym_mapping(names):
    """Build a mapping from synonyms to canonical names."""
    mapping = {}
    for name in tqdm(names, desc="Building synonym mapping"):
        canonical, synonyms = get_pubchem_data(name)
        if canonical:
            for syn in synonyms:
                mapping[syn.lower()] = canonical
    return mapping

def fuzzy_lookup(name, mapping, threshold=85):
    """Find closest match in mapping using fuzzy matching."""
    if not mapping:
        return name
    match, score = process.extractOne(name.lower(), mapping.keys())
    if score >= threshold:
        return mapping[match]
    return name

def normalize_names(df, column_name, mapping):
    """Replace excipient names with canonical names, using fuzzy matching."""
    df['CanonicalName'] = df[column_name].apply(
        lambda x: mapping.get(str(x).lower(), fuzzy_lookup(str(x), mapping))
    )
    return df

# === MAIN SCRIPT ===
# Load your file
input_file = "excipients.xlsx"  # Your uploaded file
df = pd.read_excel(input_file)

# Change 'Excipient' below to your actual column name if different
column_name = df.columns[0]  # First column in your file

# Get unique names to query
unique_names = df[column_name].dropna().unique()

# Build synonym mapping from PubChem
mapping = build_synonym_mapping(unique_names)

# Normalize dataset with fuzzy matching
df_cleaned = normalize_names(df, column_name, mapping)

# Save to new Excel
df_cleaned.to_excel("excipients_canonical.xlsx", index=False)

print("✅ Done! Cleaned list saved as 'excipients_canonical.xlsx'")


Building synonym mapping: 100%|████████████████████████████████████████████████████████| 94/94 [02:23<00:00,  1.53s/it]


✅ Done! Cleaned list saved as 'excipients_canonical.xlsx'


In [ ]:
#################################################

In [ ]:
import csv
import pubchempy as pcp
from tqdm import tqdm

# Input and output file paths
input_file = 'excipients.csv'
output_file = 'normalized_excipients_with_ids.csv'

# Read excipient names
with open(input_file, newline='', encoding='ISO-8859-1') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    excipients = [row[0].strip() for row in reader if row]

# Prepare output data
results = []

for name in tqdm(excipients, desc= "Get each excipient : "):
    try:
        compounds = pcp.get_compounds(name, 'name')
        if compounds:
            compound = compounds[0]
            cid = compound.cid
            synonyms = compound.synonyms or []
            canonical_title = synonyms[0] if synonyms else name

            # Attempt to extract MeSH ID from synonyms
            mesh_id = ''
            for syn in synonyms:
                if 'MeSH' in syn or syn.startswith('MeSH'):
                    mesh_id = syn
                    break

            all_terms = set(synonyms)
            all_terms.add(name)

            for term in all_terms:
                results.append([term, canonical_title, cid, mesh_id])
        else:
            results.append([name, 'not found', '', ''])
    except Exception:
        results.append([name, 'not found', '', ''])

# Write results to output CSV
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Synonym or MeSH Term', 'Unique Title', 'PubChem CID', 'MeSH ID'])
    writer.writerows(results)

print(f"Normalized excipients with IDs saved to {output_file}")


In [ ]:
import csv
import pubchempy as pcp

input_file = 'excipients.csv'
output_file = 'updated_excipients_table.csv'

updated_rows = []

with open(input_file, newline='', encoding='ISO-8859-1') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        excipient = row['Excipients'].strip()
        title = row['Title'].strip()
        cid = row['CID'].strip()

        if not cid:
            compounds = pcp.get_compounds(excipient, 'name')
            if compounds:
                compound = compounds[0]
                row['Title'] = compound.iupac_name or compound.synonyms[0]
                row['CID'] = str(compound.cid)
        updated_rows.append(row)

# Write updated table
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Excipients', 'Title', 'CID']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(updated_rows)

print(f"Updated table saved to {output_file}")


In [ ]:
import pubchempy as pcp

results = pcp.get_compounds("Glucose", "name")
print(results)
print(results.name)